Username: dscc440bot
Password: si.VbFxBHy559Jn

Documentation:
https://praw.readthedocs.io/en/latest/index.html

In [9]:
# Create read-only reddit instance
import praw

# This should be moved to a private praw.ini file at some point
reddit = praw.Reddit(
 client_id="kuf-S39ElN0SpA",
 client_secret="XVSM3GfyGRbhbdeXh-s3AtNqnP2FCg",
 user_agent="pc:attributes.from.punctuation:v1.0.0 (by u/dscc440bot)"
)

In [ ]:
#Test code
# Submission urls to pull top level comments from
sample_submission_urls = [
    "https://www.reddit.com/r/AskReddit/comments/9phld1/serious_reddit_what_is_your_age_and_what_problem/"
]

In [ ]:
#Test code
for url in sample_submission_urls:
    #generate list of top level comments from each submission
    comments = reddit.submission(url=url).comments
    for comment in comments:
        if type(comment) is praw.models.MoreComments:  # Comments truncate into MoreComments object. This can be expanded manually
            len(comment.comments())
        else:
            print(comment.body)

In [ ]:
#Test code
# Subreddits to pull submissions from 
sample_subreddit_names = [
    "relationships",
    "relationship_advice",
    "RoastMe",
    "RateMe",
    "amiugly",
    "AmItheAsshole"
]

for sub_name in sample_subreddit_names:
    print("###" + sub_name)
    #generate list of top posts from each subreddit
    posts = reddit.subreddit(sub_name).top("all")
    for post in posts:
            print(post.title)

In [15]:
###Main method

import csv

#list of submissions with demographic-rich top comments
submission_urls = [
    "https://www.reddit.com/r/AskReddit/comments/9phld1/serious_reddit_what_is_your_age_and_what_problem/"
]
#list of subreddits with demographic-rich submissions
subreddit_names = [
   # "relationships",
   # "relationship_advice",
   # "RoastMe",
   # "RateMe",
   # "amiugly",
   # "AmItheAsshole"
]

#initialize dataset (list of dictionaries)
dataset = []

#iterate through each list, generating data entries for each item and adding to dataset
# first: submissions with demographic-rich top comments
for url in submission_urls:
    #generate list of top level comments from each submission
    comments = reddit.submission(url=url).comments
    for comment in comments:
        if type(comment) is praw.models.MoreComments:  # Comments truncate into MoreComments object. This can be expanded manually
            print("MoreComments object found")
        else:
            entity_row = generate_data_entry(comment, 1606780800)  # ref date of 12/01/2020 midnight
            if entity_row:
                print(comment.permalink)
                dataset.append(entity_row)
                #print(entity_row)

# second: subreddits with demographic-rich post titles
for sub_name in subreddit_names:
    #generate list of top posts from each subreddit
    posts = reddit.subreddit(sub_name).top("all")
    for post in posts:
        entity_row = generate_data_entry(post, 1606780800)  # ref date of 12/01/2020 midnight
        if entity_row:
            print(post.permalink)
            dataset.append(entity_row)
            #print(entity_row)

Account not valid (probably suspended).


NameError: name 'data' is not defined

In [10]:
#save dataset
#code adapted from https://stackoverflow.com/questions/3086973/how-do-i-convert-this-list-of-dictionaries-to-a-csv-file
keys = dataset[0].keys()
with open('stats.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(dataset)

IndexError: list index out of range

In [11]:
year_utc = 31536000

def generate_data_entry(text_instance, current_utc):
    '''
    Generate an entry to be added to the final dataset. Entry will be created from text_instance object. Text_instance can be either a submision or comment object.
    
    Returns dictionary(?) containing redditor name, age, gender, and punctuation statistics
    Returns None if no useful data entry can be generated.
    '''
    # Check if author's account still exists
    if text_instance.author is None:
        return None
    
    text = None
    if type(text_instance) is praw.models.Submission:
        text = text_instance.title + text_instance.selftext
    elif type(text_instance) is praw.models.reddit.comment.Comment:
        text = text_instance.body
    else:
        #error
        print("Item cannot be parsed (Is not a comment or submission)")
        print(type(text_instance))

    #Find demographics
    demog = find_demographics(text)
    
    if demog is None:
        return None
    
    #Calculate current age
    if demog["age"] is not None:
        utc_at_comment = text_instance.created_utc
        years_since = int((current_utc - utc_at_comment)/year_utc)
        demog["age"] = demog["age"] + years_since
    
    # For testing, make each entity row identifiable
    demog["url"] = text_instance.permalink
    
    #If demographics are found, calculate puncuation statistics
    p_stats = calculate_punctuation_stats(text_instance.author,1596240000,1604188800)  #08/01/2020-11/01/2020
    
    if p_stats is None:
        return None
    
    #Format and return data entry
    return {**demog, **p_stats}

In [12]:
import re

def find_demographics(text):
    '''
    Find the age and gender information contained in the text parameter.
    
    Returns a dictionary of the age and gender information contained in the text. If a feature cannot be determined, its dictionary value will be None.
    '''
    out = {}
    out["age"] = None
    out["gender"] = None
    
    ### find demographic info
    # Check for ##Gender format with or without space or coma (Ex. 32M or 28 F or 23,M or 54, F)
    match = re.findall(r"\b(\d{2}),? ?\/?(M|m|Male|male|F|f|Female|female)\b",text)
    if len(match) == 1:
        out["age"] = int(match[0][0])
        g = match[0][1]
        if g in "F f Female female":
            out["gender"] = "Female"
        else:
            out["gender"] = "Male"
        return out
    elif len(match) > 1:  # Catches cases where two people are identified (Ex. My wife (23f) and I (25M))
        # first, focus matched text to more specific criteria including identifying pronoun
        match = re.findall(r"(?:I|i|me|Me|my|My|I'[a-zA-Z]+|i'[a-zA-Z]+)\s?[\(\[\s](\d{2})[,\/ ]?(M|m|Male|male|F|f|Female|female)[\)\]\s]",text)
        if match:
            out["age"] = int(match[0][0])
            g = match[0][1]
            if g in "F f Female female":
                out["gender"] = "Female"
            else:
                out["gender"] = "Male"
            return out
    
    # Check for ## age format (Ex. 32 or 50)
    match = re.findall(r"\b(\d{2})[,.:-]?\b",text)
    if match:
        out["age"] = int(match[0])
        return out
    
    return None

In [13]:
import re
import itertools
import pandas as pd
from statistics import mean
from string import punctuation
from itertools import chain
import math

def calculate_punctuation_stats(redditor, start_time, end_time):
    '''
    Generate punctuation statistics from posts of redditor during the time range between start_time and end_time (both in UTC, Unix time).
    
    Returns dictionary(?) of punctuation statistics
    '''
    comment_list = []  # contains strings of comment text
    
    try:
        for comment in redditor.comments.new(limit=None):
            if comment.created_utc > start_time and comment.created_utc < end_time:  # Check if comment is within time range
                comment_list.append(comment.body)
    except:
        print("Account not valid (probably suspended).")
        
    if len(comment_list) == 0:
        return None
        
    # This function deals with reformatting of the comment-array input.

    # Concatinates all the comments into one sample of the Redditor's writing (string object).
    writing_sample = ' '.join(comment_list)
    
    # Remove new line
    writing_sample = writing_sample.replace('\n', " ")

    # Removing all non-ASCII letters
    writing_sample = writing_sample.encode("ascii", "ignore").decode()
    
    # Remove URLs from comments
    writing_sample = re.sub(r'\(https?:\/\/.*?\)', '', writing_sample)

    # Now, we need to check for instances of elipses and assign them to a new symbol--less confusion with periods, easier analysis.
    # The tilde will represent an elipse from here on out.
    writing_sample = writing_sample.replace('...', '~')

    # We will loop over every character in the string and extract the punctuation, appending each mark to a list.
    # We will keep this punctuation sequence in both list and string form for further analysis.
    marks = ['.',',',':',';','?','!','-','(',')','"',"'",'~']
    marks_str = ''.join(map(str, marks))

    punct_list = []

    for character in writing_sample:
        if character in marks:
            punct_list.append(character)

    punct_str = ''.join(punct_list)
    
    if len(punct_list) == 0:
        return None

    # Now, we want to get our input in the following format..[2, '!', 3, '.'].
    split_by_punc = (x.split() for x in re.split('[' + marks_str + ']', writing_sample))
    wrds_btwn = list(map(len, split_by_punc))
    woven = list(chain.from_iterable(zip(wrds_btwn, punct_list)))
    
    # Develop punctuation stats for comments_text
    # Generation of F1, relative frequency of each punctuation mark of interest.
    num_period = punct_str.count('.')
    num_comma = punct_str.count(',')
    num_colon = punct_str.count(':')
    num_semicol = punct_str.count(';')
    num_question = punct_str.count('?')
    num_exclaim = punct_str.count('!')
    num_dash = punct_str.count('-')
    num_left_paren = punct_str.count('(')
    num_right_paren = punct_str.count(')')
    num_sing_quote = punct_str.count("'")
    num_doub_quote = punct_str.count('"')
    num_elipse = punct_str.count('~')

    mark_counts = [num_period, num_comma, num_colon, num_semicol, num_question, num_exclaim, num_dash, num_left_paren, num_right_paren, num_sing_quote, num_doub_quote, num_elipse] 
    F1 = [x / len(punct_list) for x in mark_counts]
    
    # Generation of F2 and F3, conditional/joint probability of observing two punctuation marks in succession.
    F2_dict = {}
    F3_dict = {}
    successive_pairs = []
    
    combos = list(itertools.product(marks_str, marks_str))
    indices = list(range(len(marks_str)))
    index_combos = list((itertools.product(indices, indices)))
    index_strings = [[str(x) for x in tup] for tup in index_combos]
    final_ind = list(map(' '.join, data)) 
    #combos = list(itertools.product(marks_str, marks_str))
    #indices = list(range(marks_str)) 
    #index_combos = list(itertools.product(indices, indices)) 

    for i in combos:
        successive_pairs.append(''.join(i))
        
    zipped = list(zip(final_ind, successive_pairs)) 

    for (index, pair) in zipped:
        numerator = punct_str.count(pair)
        denominator = punct_str.count(pair[0])

        if numerator != 0:
            F2_dict['Conditional Prob' + ' ' + index + ' ' + pair + ' '] = (numerator/denominator)
            F3_dict['Joint Prob' + ' ' + index + ' ' + pair + ' '] = (numerator/len(punct_list))
        else:
            F2_dict['Conditional Prob' + ' ' + index + ' ' + pair + ' '] = 0
            F3_dict['Joint Prob' + ' ' + index + ' ' + pair + ' '] = 0
            
    # Generation of f4, redditor's average sentence length in words.
    sent_lens = []
    max_sents = 200
    regex = "\w+('\w+)?(?<!('s))"
    sentences = split_into_sentences(writing_sample)

    for sentence in sentences:
        sent_lens.append(len(re.findall(regex, sentence)))

    if len(sent_lens) == 0:
        sent_lens.append(0)
    
    f4 = mean(sent_lens)

    # f4 as a probability distribution of sentence length--as it is in Darmon's work.
    # Cap sentence length at 199 words.
    F4 = [0]*max_sents
    
    sent_lens = [x if x < max_sents else max_sents-1 for x in sent_lens]

    for i in sent_lens:
        F4[i] += 1

    F4 = [x / len(sent_lens) for x in F4]
    
    # Generation of f5, redditor's average number of words between successive punctuation marks.
    if len(wrds_btwn) == 0:
        wrds_btwn.append(0)
        
    f5 = mean(wrds_btwn)

    # f5 as a probability distribution of number of words between successive marks--as it is in Darmon's work.
    # Cap number of in-between words at 39.
    max_wrds_btwn = 40
    wrds_btwn = [x if x < max_wrds_btwn else max_wrds_btwn-1 for x in wrds_btwn]
    F5 = [0]*max_wrds_btwn

    for i in wrds_btwn:
        F5[i] += 1

    F5 = [x / len(wrds_btwn) for x in F5]
    
    # Generation of f6, ratio of punctuation to words.
    total_punc = len(punct_list)
    total_words = len(re.findall(regex, writing_sample))

    f6 = total_punc/total_words
    
    # Output to dataframe.
    # An uppercase F signals that the feature was mimicked directly from Darmon's work.
    F1_dict_keys = ['Period Freq', 'Comma Freq', 'Colon Freq', 'Semicol Freq', 'Question Freq', 'Exclaim Freq', 'Dash Freq', 'L. Paren Freq', 'R. Paren Freq', 'S. Quote Freq', 'D. Quote Freq', 'Elipse Freq']
    F1_dict = {F1_dict_keys[i]: F1[i] for i in range(len(F1_dict_keys))} 

    F4_dict = {'Prob Sentence Len:' + ' ' + str(i): F4[i] for i in range(len(F4))}
    F5_dict = {'Prob' + ' ' + str(i) + ' ' + 'Words Between Punc': F5[i] for i in range(len(F5))}

    f4_dict = {'Avg Sentence Len': f4}
    f5_dict = {'Avg Wrds Btwn Punc': f5}
    f6_dict = {'Punc Ratio': f6}
    
    # Concatinate all the keys into a single list.
    master = {**F1_dict, **F2_dict, **F3_dict, **F4_dict, **F5_dict, **f4_dict, **f5_dict, **f6_dict}
    
    return master

In [14]:
# Helper function which splits text input into individual sentances. Used in generation of f4.

acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
alphabets= "([A-Za-z])"
digits = "([0-9])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text) 
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences